In [8]:
!pip install dataset
!pip install funcy
!pip install pyquery

import csv
import errno
import itertools
import os
import time

from bs4 import BeautifulSoup
import click
import dataset
import funcy as fy
import requests

from pyquery import PyQuery
import time
import traceback

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
%matplotlib inline

from datetime import datetime
import time


In [17]:
state_id = 0
gender = 0
number_of_results = 40000
headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'en-US,en;q=0.8',
        'Cache-Control': 'max-age=0',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Origin': 'http://registration.baa.org',
        'Referer': 'http://registration.baa.org/2022/cf/Public/iframe_ResultsSearch.cfm?mode=results',
    }
params = {
        'mode': 'results',
        'criteria': '',
        'StoredProcParamsOn': 'yes',
        'VarGenderID': gender,
        'VarBibNumber': '',
        'VarLastName': '',
        'VarFirstName': '',
        'VarStateID': state_id,
        'VarCountryOfResID': 0,
        'VarCountryOfCtzID': 0,
        'VarReportingSegID': 1,
        'VarAwardsDivID': 0,
        'VarQualClassID': 0,
        'VarCity': '',
        'VarTargetCount': number_of_results,
        'records': 100,
        'headerexists': 'Yes',
        'bordersize': 0,
        'bordercolor': '#ffffff',
        'rowcolorone': '#FFCC33',
        'rowcolortwo': '#FFCC33',
        'headercolor': '#ffffff',
        'headerfontface': 'Verdana,Arial,Helvetica,sans-serif',
        'headerfontcolor': '#004080',
        'headerfontsize': '12px',
        'fontface': 'Verdana,Arial,Helvetica,sans-serif',
        'fontcolor': '#000099',
        'fontsize': '10px',
        'linkfield': 'FormattedSortName',
        'linkurl': 'OpenDetailsWindow',
        'linkparams': 'RaceAppID',
        'queryname': 'SearchResults',
        'tablefields': 'FullBibNumber,FormattedSortName,AgeOnRaceDay,GenderCode,'
                       'City,StateAbbrev,CountryOfResAbbrev,CountryOfCtzAbbrev,'
                       'DisabilityGroup',
    }

In [ ]:
results = []

def get_results_url(year):
    return f'http://registration.baa.org/{year}/cf/Public/iframe_ResultsSearch.cfm'

for year in range(2022, 2023):  # Adjust the range for desired years
    click.echo(f"Scraping data for year: {year}")
    for page_number, start in enumerate(itertools.count(1, 100)):
        time.sleep(1.0)  # Be kind to the server

        click.echo('page %d of %d' % (page_number + 1, number_of_results/100))
        response = requests.post(
            'http://registration.baa.org/2022/cf/Public/iframe_ResultsSearch.cfm',
            headers=headers,
            params=params,
            data={'start': start, 'next': 'Next 25 Records'},
        )
        #peopleList = extract_data(response.content)
        soup = BeautifulSoup(response.text, 'lxml')
        table = soup.find("table", attrs={"class": "tablegrid_table"})
        rows = table.findAll("tr")
        for row in rows:
            a = [t.text.strip() for t in row.findAll("td")][0:]
            #Don't store lines without data
            if len(a) > 0 and a != [''] and a !=['',''] and a != ['', '', '']: 
                results.append(a)

        # Only yield if there actually are results. Just found this random
        # tr_header thing in the HTML of the pages that have results, but not
        # empty results pages.
        if 'tr_header' in response.text:
            (page_number, response.text)
        else:
            assert 'Next 25 Records' not in response.text
            click.echo('  No results found.')
            break

        # No more pages!
        if 'Next 25 Records' not in response.text:
            break

Scraping data for year: 2022
page 1 of 400
page 2 of 400
page 3 of 400
page 4 of 400
page 5 of 400
page 6 of 400
page 7 of 400
page 8 of 400
page 9 of 400
page 10 of 400
page 11 of 400
page 12 of 400
page 13 of 400
page 14 of 400
page 15 of 400
page 16 of 400
page 17 of 400
page 18 of 400
page 19 of 400
page 20 of 400
page 21 of 400
page 22 of 400
page 23 of 400
page 24 of 400
page 25 of 400
page 26 of 400
page 27 of 400
page 28 of 400
page 29 of 400
page 30 of 400
page 31 of 400
page 32 of 400
page 33 of 400
page 34 of 400
page 35 of 400
page 36 of 400
page 37 of 400
page 38 of 400
page 39 of 400
page 40 of 400
page 41 of 400
page 42 of 400
page 43 of 400
page 44 of 400
page 45 of 400
page 46 of 400
page 47 of 400
page 48 of 400
page 49 of 400
page 50 of 400
page 51 of 400
page 52 of 400
page 53 of 400
page 54 of 400
page 55 of 400
page 56 of 400
page 57 of 400
page 58 of 400
page 59 of 400


In [ ]:
data = []
for i, result in enumerate(results):
    if i%4 == 0:
        data.append(results[i] + results[i+1][1:])

In [ ]:
columns = ['Bib', 'Name', 'Age', 'M/F', 'City', 'State', 'Country', 'Citizen', '', '5K', '10K', '15K', '20K', 'Half',
          '25K', '30K', '35K', '40K', 'Pace', 'Proj Time', 'Official Time', 'Overall', 'Gender', 'Division']


In [ ]:
filename = 'marathon_results_2022_V2.csv'

In [ ]:
import pandas as pd
df1 = pd.DataFrame(data, columns=columns)
df1.to_csv(filename, index=False)

In [ ]:
df1
